Installing necessary modules.



In [ ]:
!pip install transformers #Necessary for the ipa transcription of the words
!pip install panphon
!pip install researchpy

In [ ]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from google.colab import drive
import random
from transformers import T5ForConditionalGeneration, AutoTokenizer
import panphon
import pickle
from math import sqrt
from scipy.stats import ttest_rel, ttest_ind
from researchpy import ttest
import torch
import statistics
from sklearn.utils import class_weight

To be able to access the data files that are going to be used in this script. All of them are uploaded in my Drive.

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Reading the dataset that has the multilingual WordNet items aligned according to the sense to a dataframe.

In [ ]:
data = open("/content/drive/My Drive/MultilingualData.csv", "r", encoding="utf-8")
data = pd.read_csv(data)
data = data.iloc[:,1:]
print(data)

            id                                           word_eng  \
0   00029933-a  avaricious,covetous,grabby,grasping,greedy,pre...   
1   00079485-a                                            fearful   
2   00080357-a  panicky,panicked,panic-stricken,panic-struck,t...   
3   00081671-a                                  unafraid,fearless   
4   00089550-a  annoying,bothersome,galling,irritating,nettles...   
..         ...                                                ...   
94  07554856-n                                            empathy   
95  07555014-n                                         enthusiasm   
96  07555184-n                eagerness,avidity,avidness,keenness   
97  07555402-n                             ardor,ardour,elan,zeal   
98  07555647-n                                         exuberance   

                                             word_fin  \
0                   ahne,rahanahne,himokas,himoitseva   
1                            pelokas,peloissaan oleva   


Filtering out the positive words in the dataset to form the positive words dataset.

In [ ]:
positive_data = data[data['categ'].isin(['joy', 'love', 'affection', 'liking', 'enthusiasm', 'gratitude', 'self-pride', 'levity', 'calmness', 'fearlessness', 'positive-expectation', 'positive-fear', 'positive-hope'])]
print(positive_data)
print(positive_data.shape)

            id                                           word_eng  \
3   00081671-a                                  unafraid,fearless   
8   00200777-r                                  eagerly,thirstily   
9   00291471-a                    brotherly,brotherlike,fraternal   
10  00302951-a       placid,quiet,still,tranquil,smooth,unruffled   
12  00885695-a                                       enthusiastic   
13  01146493-a                                  grateful,thankful   
14  01148283-a                                              happy   
15  01361414-a                                               glad   
17  02080577-a                                       satisfactory   
19  02678438-v                      concern,interest,occupy,worry   
20  07491981-n                      gusto,relish,zest,zestfulness   
21  07497473-n                                             liking   
22  07497797-n                          fondness,fancy,partiality   
23  07500042-n                    

In [ ]:
positive_data['categ'] = positive_data['categ'].replace(['joy', 'love', 'affection', 'liking', 'enthusiasm', 'gratitude', 'calmness', 'fearlessness', 'positive-expectation', 'positive-fear', 'positive-hope'], [(int(i+9)) for i in range(11)])
#positive_data['categ'] = positive_data['categ'].replace(['joy', 'love', 'affection', 'liking', 'enthusiasm', 'gratitude', 'calmness', 'fearlessness', 'positive-expectation', 'positive-fear', 'positive-hope'], [int(3), int(4), int(4), int(4), int(3), int(4), int(3), int(3), int(3), int(3), int(3)])
print(positive_data)

            id                                           word_eng  \
3   00081671-a                                  unafraid,fearless   
8   00200777-r                                  eagerly,thirstily   
9   00291471-a                    brotherly,brotherlike,fraternal   
10  00302951-a       placid,quiet,still,tranquil,smooth,unruffled   
12  00885695-a                                       enthusiastic   
13  01146493-a                                  grateful,thankful   
14  01148283-a                                              happy   
15  01361414-a                                               glad   
17  02080577-a                                       satisfactory   
19  02678438-v                      concern,interest,occupy,worry   
20  07491981-n                      gusto,relish,zest,zestfulness   
21  07497473-n                                             liking   
22  07497797-n                          fondness,fancy,partiality   
23  07500042-n                    

<ipython-input-28-1228ae6dc873>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  positive_data['categ'] = positive_data['categ'].replace(['joy', 'love', 'affection', 'liking', 'enthusiasm', 'gratitude', 'calmness', 'fearlessness', 'positive-expectation', 'positive-fear', 'positive-hope'], [(int(i+9)) for i in range(11)])


Filtering out the negative words in the dataset to form the negative words dataset.

In [ ]:
negative_data = data[data['categ'].isin(['negative-fear', 'sadness', 'general-dislike', 'ingratitude', 'shame', 'compassion', 'humility', 'despair', 'anxiety'])]
print(negative_data)
print(negative_data.shape)

            id                                           word_eng  \
0   00029933-a  avaricious,covetous,grabby,grasping,greedy,pre...   
1   00079485-a                                            fearful   
2   00080357-a  panicky,panicked,panic-stricken,panic-struck,t...   
4   00089550-a  annoying,bothersome,galling,irritating,nettles...   
5   00113818-a                                              angry   
7   00193480-a       atrocious,frightful,horrifying,horrible,ugly   
11  00339941-a                         diffident,shy,timid,unsure   
16  01364008-a                                          sorrowful   
18  02678438-v                      concern,interest,occupy,worry   
25  07502669-n                        antipathy,aversion,distaste   
26  07502980-n                     contempt,disdain,scorn,despite   
27  07503430-n  abhorrence,abomination,detestation,execration,...   
29  07505207-n                          solicitude,solicitousness   
30  07506382-n    cruelty,merciles

In [ ]:
negative_data['categ'] = negative_data['categ'].replace(['negative-fear', 'sadness', 'general-dislike', 'shame', 'compassion', 'humility', 'despair', 'anxiety', 'daze'], [int(i) for i in range(9)])
#negative_data['categ'] = negative_data['categ'].replace(['negative-fear', 'sadness', 'general-dislike', 'shame', 'compassion', 'humility', 'despair', 'anxiety'], [int(0), int(1), int(2), int(2), int(4), int(2), int(1), int(0)])
print(negative_data)

            id                                           word_eng  \
0   00029933-a  avaricious,covetous,grabby,grasping,greedy,pre...   
1   00079485-a                                            fearful   
2   00080357-a  panicky,panicked,panic-stricken,panic-struck,t...   
4   00089550-a  annoying,bothersome,galling,irritating,nettles...   
5   00113818-a                                              angry   
7   00193480-a       atrocious,frightful,horrifying,horrible,ugly   
11  00339941-a                         diffident,shy,timid,unsure   
16  01364008-a                                          sorrowful   
18  02678438-v                      concern,interest,occupy,worry   
25  07502669-n                        antipathy,aversion,distaste   
26  07502980-n                     contempt,disdain,scorn,despite   
27  07503430-n  abhorrence,abomination,detestation,execration,...   
29  07505207-n                          solicitude,solicitousness   
30  07506382-n    cruelty,merciles

<ipython-input-30-7fa0dc33fad1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_data['categ'] = negative_data['categ'].replace(['negative-fear', 'sadness', 'general-dislike', 'shame', 'compassion', 'humility', 'despair', 'anxiety', 'daze'], [int(i) for i in range(9)])


Separating the synsets as the ones that will be used in the training and test.

In [ ]:
def dataset_sep():
  positive_negative_id = positive_data['id'].values.tolist() + negative_data['id'].values.tolist()
  train_seed = set(random.sample(positive_negative_id, int(round(len(positive_negative_id)*80/100, 0)))); print("80% =", len(train_seed))
  test_seed = set(positive_negative_id)-train_seed; print("20% =", len(test_seed))
  stimuli_df = pd.concat([positive_data, negative_data])
  train =  stimuli_df[stimuli_df["id"].isin(train_seed)]
  test = stimuli_df[stimuli_df["id"].isin(test_seed)]
  return train, test

Merging all the stimuli into a single dataframe to prepare for the separation of the training and test data.

In [ ]:
'''stimuli_df = pd.concat([positive_data, negative_data])
print(stimuli_df)'''

'stimuli_df = pd.concat([positive_data, negative_data])\nprint(stimuli_df)'

Separating the training and test data depending on the separation that is conducted above with ids.

In [ ]:
'''train =  stimuli_df[stimuli_df["id"].isin(train_seed)]
test = stimuli_df[stimuli_df["id"].isin(test_seed)]'''

'train =  stimuli_df[stimuli_df["id"].isin(train_seed)]\ntest = stimuli_df[stimuli_df["id"].isin(test_seed)]'

In [ ]:
'''print(train)'''

'print(train)'

In [ ]:
'''print(test)'''

'print(test)'

Setting up the model for generating the IPA versions of the words from experimental languages.

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('charsiu/g2p_multilingual_byT5_tiny_16_layers_100')
tokenizer = AutoTokenizer.from_pretrained('google/byt5-small')

A function that transliterates a word into IPA.

In [ ]:
def ipa_translator(word,lang):
  word = '<'+ lang +'>'+ word
  out = tokenizer(word,padding=True,add_special_tokens=False,return_tensors='pt')
  preds = model.generate(**out,num_beams=1,max_length=15)
  ipa = tokenizer.batch_decode(preds.tolist(),skip_special_tokens=True)
  return ipa[0]

A code that goes through all the items in the training dataframe and transliterates all of them into IPA then converts them into phonological feature vectors. The final stimuli that will be used in training LSTMs are kept in a new dataframe named *train_phon*.

In [ ]:
def trainset_prep(train):
  train_phon = train.copy()
  langs = list(train_phon.columns)[1:-1]
  ft = panphon.FeatureTable()
  phon_features = ['syl', 'son', 'cons', 'cont', 'delrel', 'lat', 'nas', 'strid', 'voi', 'sg', 'cg', 'ant', 'cor', 'distr', 'lab', 'hi', 'lo', 'back', 'round', 'velaric', 'tense', 'long']
  training_w = {}
  for lang in langs:
    training_w[lang] = []
    for index, row in train_phon[lang].items():
      word = ''
      if ',' in row:
        r = row.split(',')
        random.shuffle(r)
        for w in r:
          if w not in training_w[lang]:
            word = w
            break
      else:
        if row not in training_w[lang]:
          word = row
      if word != '':
        training_w[lang].append(word)
        ipa_word = ipa_translator(word, lang[-3:])
        phon_vec = ft.word_array(phon_features, ipa_word)[0]
        row = phon_vec
      else:
        row = np.nan
      train_phon.at[index, lang] = row
  train_phon = train_phon.dropna()
  return train_phon, training_w

A code that goes through all the items in the test dataframe and transliterates all of them into IPA then converts them into phonological feature vectors. The final stimuli that will be used in testing the LSTMs are kept in a new dataframe named *test_phon*.

In [ ]:
def testset_prep(test, training_w):
  test_phon = test.copy()
  langs = list(test_phon.columns)[1:-1]
  ft = panphon.FeatureTable()
  phon_features = ['syl', 'son', 'cons', 'cont', 'delrel', 'lat', 'nas', 'strid', 'voi', 'sg', 'cg', 'ant', 'cor', 'distr', 'lab', 'hi', 'lo', 'back', 'round', 'velaric', 'tense', 'long']
  for lang in langs:
    for index, row in test[lang].items():
      if ',' in row:
        r = row.split(',')
        word = ''
        random.shuffle(r)
        for w in r:
          if w not in training_w[lang]:
            word = w
            break
      else:
        if row not in training_w[lang]:
          word = row
      if word != '':
        training_w[lang].append(word)
        ipa_word = ipa_translator(word, lang[-3:])
        phon_vec = ft.word_array(phon_features, ipa_word)[0]
        row = phon_vec
      else:
        row = np.nan
      test_phon.at[index, lang] = row
  test_phon = test_phon.dropna()
  return test_phon

In [ ]:
'''train_phon = train_phon.replace(['positive-emotion','negative-emotion'], [float(1),float(0)])
print(train_phon)'''

"train_phon = train_phon.replace(['positive-emotion','negative-emotion'], [float(1),float(0)])\nprint(train_phon)"

Replacing the *positive* and *negative* emotion tags with 1 and 0 respectively for making it easy for training and testing.

In [ ]:
'''test_phon = test_phon.replace(['positive-emotion','negative-emotion'], [float(1),float(0)])
print(test_phon)'''

"test_phon = test_phon.replace(['positive-emotion','negative-emotion'], [float(1),float(0)])\nprint(test_phon)"

A function that takes a language as an argument and trains an LSTM with the phonological vectors and the emotional valence of the words in languages other than the language in the argument. Then, it evaluates the predictions of the LSTM for the emotional valence of the words in the test language.

In [ ]:
def multilingual_network(language):
  ls = [col for col in list(test_phon.columns)[1:-1] if col != language]
  input_train = [train_phon[l] for l in ls]
  input_train = pd.concat(input_train)
  input_train = input_train.to_numpy()
  target_train = [train_phon['categ'] for i in range(6)]
  input_train = np.array([np.array(val) for val in input_train])
  target_train = pd.concat(target_train)
  #class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(target_train), y=target_train)
  #print(class_weights)
  target_train = target_train.to_numpy()
  target_train = np.array([np.array(val) for val in target_train])
  target_train = np.reshape(target_train, (input_train.shape[0],1))
  print("input_train shape =", input_train.shape)
  print("target_train shape =", target_train.shape)
  input_test = test_phon[language].to_numpy()
  input_test = np.array([np.array(val) for val in input_test])
  target_test = test_phon['categ'].to_numpy()
  target_test = np.array([np.array(val) for val in target_test])
  target_test = np.reshape(target_test, (input_test.shape[0],1))
  print('input_test shape =', input_test.shape)
  print('target_test shape =', target_test.shape)
  print("Starting to train LSTM - MULTILINGUAL CONDITION")
  model = Sequential()
  model.add(Dense(50, input_shape=(input_train.shape[1],), activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  #learning_rate = 0.0001
  #optimizer = Adam(learning_rate=learning_rate)
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit(input_train, target_train, epochs=50, batch_size=5, shuffle=True)
  loss_m, accuracy_m = model.evaluate(input_test, target_test)
  return loss_m, accuracy_m

A function that is the baseline for the *multilingual_network* function. It trains an LSTM in the same way but it shuffles the *target_train* array for the baseline condition.

In [ ]:
def random_network(language):
  ls = [col for col in list(test_phon.columns)[1:-1] if col != language]
  input_train = [train_phon[l] for l in ls]
  input_train = pd.concat(input_train)
  input_train = input_train.to_numpy()
  target_train = [train_phon['categ'] for i in range(6)]
  input_train = np.array([np.array(val) for val in input_train])
  target_train = pd.concat(target_train)
  #class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(target_train), y=target_train)
  target_train = target_train.to_numpy()
  target_train = np.array([np.array(val) for val in target_train])
  target_train = np.reshape(target_train, (input_train.shape[0],1))
  print("input_train shape =", input_train.shape)
  print("target_train shape =", target_train.shape)
  input_test = test_phon[language].to_numpy()
  input_test = np.array([np.array(val) for val in input_test])
  target_test = test_phon['categ'].to_numpy()
  target_test = np.array([np.array(val) for val in target_test])
  target_test = np.reshape(target_test, (input_test.shape[0],1))
  print('input_test shape =', input_test.shape)
  print('target_test shape =', target_test.shape)
  print("Starting to train LSTM - MULTILINGUAL CONDITION")
  np.random.shuffle(target_train)
  model_r = Sequential()
  model_r.add(Dense(50, input_shape=(input_train.shape[1],), activation='relu'))
  model_r.add(Dense(1, activation='sigmoid'))
  #learning_rate = 0.0001
  #optimizer = Adam(learning_rate=learning_rate)
  model_r.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model_r.fit(input_train, target_train, epochs=50, batch_size=5, shuffle=True)
  loss_r, accuracy_r = model_r.evaluate(input_test, target_test)
  return loss_r, accuracy_r

In [ ]:
acc = {'eng':[], 'fin':[], 'jpn':[], 'slv':[], 'spa':[], 'tha':[], 'zsm':[]}
s_acc = {'eng':[], 'fin':[], 'jpn':[], 'slv':[], 'spa':[], 'tha':[], 'zsm':[]}
for i in range(20):
  train, test = dataset_sep()
  train_phon, training_w = trainset_prep(train)
  test_phon = testset_prep(test, training_w)
  for l in list(test_phon.columns)[1:-1]:
    print("\n\n\n")
    print(l[-3:].upper(), "\n")
    loss, accuracy = multilingual_network(l)
    s_loss, s_accuracy = random_network(l)
    print('Accuracy of the model for ', l[-3:], ' = ', accuracy)
    print('Accuracy of the random model for ', l[-3:], ' = ', s_accuracy)
    acc[l[-3:]].append(accuracy)
    s_acc[l[-3:]].append(s_accuracy)

80% = 73
20% = 19




ENG 

input_train shape = (390, 22)
target_train shape = (390, 1)
input_test shape = (15, 22)
target_test shape = (15, 1)
Starting to train LSTM - MULTILINGUAL CONDITION
Epoch 1/50
78/78 [==============================] - 1s 2ms/step - loss: -12.4349 - accuracy: 0.0692
Epoch 2/50
78/78 [==============================] - 0s 2ms/step - loss: -55.2380 - accuracy: 0.0615
Epoch 3/50
78/78 [==============================] - 0s 2ms/step - loss: -136.1808 - accuracy: 0.0615
Epoch 4/50
78/78 [==============================] - 0s 2ms/step - loss: -265.8901 - accuracy: 0.0615
Epoch 5/50
78/78 [==============================] - 0s 2ms/step - loss: -445.8606 - accuracy: 0.0615
Epoch 6/50
78/78 [==============================] - 0s 2ms/step - loss: -682.9488 - accuracy: 0.0615
Epoch 7/50
78/78 [==============================] - 0s 2ms/step - loss: -970.0386 - accuracy: 0.0615
Epoch 8/50
78/78 [==============================] - 0s 2ms/step - loss: -1312.1584 - accuracy: 0.0615
E

1/1 [==============================] - 0s 220ms/step - loss: -47781.9258 - accuracy: 0.0667
input_train shape = (390, 22)
target_train shape = (390, 1)
input_test shape = (15, 22)
target_test shape = (15, 1)
Starting to train LSTM - MULTILINGUAL CONDITION
Epoch 1/50
78/78 [==============================] - 1s 3ms/step - loss: -11.1453 - accuracy: 0.0667
Epoch 2/50
78/78 [==============================] - 0s 3ms/step - loss: -50.4729 - accuracy: 0.0615
Epoch 3/50
78/78 [==============================] - 0s 2ms/step - loss: -131.5274 - accuracy: 0.0615
Epoch 4/50
78/78 [==============================] - 0s 3ms/step - loss: -268.5231 - accuracy: 0.0615
Epoch 5/50
78/78 [==============================] - 0s 3ms/step - loss: -457.5457 - accuracy: 0.0615
Epoch 6/50
78/78 [==============================] - 0s 2ms/step - loss: -691.9490 - accuracy: 0.0615
Epoch 7/50
78/78 [==============================] - 0s 2ms/step - loss: -975.3405 - accuracy: 0.0615
Epoch 8/50
78/78 [=====================

Görüntülenen çıkış son 5000 satıra kısaltıldı.
78/78 [==============================] - 0s 2ms/step - loss: -16313.3848 - accuracy: 0.0769
Epoch 29/50
78/78 [==============================] - 0s 3ms/step - loss: -17410.0469 - accuracy: 0.0769
Epoch 30/50
78/78 [==============================] - 0s 3ms/step - loss: -18539.4102 - accuracy: 0.0769
Epoch 31/50
78/78 [==============================] - 0s 3ms/step - loss: -19692.7363 - accuracy: 0.0769
Epoch 32/50
78/78 [==============================] - 0s 3ms/step - loss: -20873.2949 - accuracy: 0.0769
Epoch 33/50
78/78 [==============================] - 0s 2ms/step - loss: -22084.2812 - accuracy: 0.0769
Epoch 34/50
78/78 [==============================] - 0s 2ms/step - loss: -23313.4727 - accuracy: 0.0769
Epoch 35/50
78/78 [==============================] - 0s 3ms/step - loss: -24582.7812 - accuracy: 0.0769
Epoch 36/50
78/78 [==============================] - 0s 2ms/step - loss: -25863.2148 - accuracy: 0.0769
Epoch 37/50
78/78 [==========

In [ ]:
for l in acc:
  print('Mean accuracy of the multilingual model in', l,' =', round(statistics.mean(acc[l]),2))
  print('Mean accuracy of the random model in', l, ' =', round(statistics.mean(s_acc[l]),2))
  t_statistic, p_value = ttest_ind(acc[l], s_acc[l])
  if p_value < 0.05:
      print("The difference in accuracies is statistically significant in", l, 'p-value = ', p_value)
  else:
      print("There is no significant difference in accuracies between the two models in", l, 'p-value = ', p_value)

accuracies = [acc[l] for l in acc]
accuracies = accuracies[0] + accuracies[1] + accuracies[2]
m_accuracies = statistics.mean(accuracies)
s_accuracies = [s_acc[l] for l in s_acc]
s_accuracies = s_accuracies[0] + s_accuracies[1] + s_accuracies[2]
m_s_accuracies = statistics.mean(s_accuracies)

print('Mean accuracy of the multilingual model: ', round(m_accuracies,2))
print('Mean accuracy of the random model: ', round(m_s_accuracies,2))
t_statistic, p_value = ttest_ind(accuracies, s_accuracies)
if p_value < 0.05:
    print("The difference in accuracies is statistically significant. P-value = ", p_value)
else:
    print("There is no significant difference in accuracies between the two models. P-value = ", p_value)

Mean accuracy of the multilingual model in eng  = 0.07
Mean accuracy of the random model in eng  = 0.07
There is no significant difference in accuracies between the two models in eng p-value =  1.0
Mean accuracy of the multilingual model in fin  = 0.07
Mean accuracy of the random model in fin  = 0.07
There is no significant difference in accuracies between the two models in fin p-value =  1.0
Mean accuracy of the multilingual model in jpn  = 0.07
Mean accuracy of the random model in jpn  = 0.07
There is no significant difference in accuracies between the two models in jpn p-value =  1.0
Mean accuracy of the multilingual model in slv  = 0.07
Mean accuracy of the random model in slv  = 0.07
There is no significant difference in accuracies between the two models in slv p-value =  1.0
Mean accuracy of the multilingual model in spa  = 0.07
Mean accuracy of the random model in spa  = 0.07
There is no significant difference in accuracies between the two models in spa p-value =  1.0
Mean accur